In [6]:
import pandas as pd
import ipywidgets as widgets
import models as m

from sqlalchemy import func
from sqlalchemy.orm import sessionmaker, relationship

import config

pd.options.display.max_rows = 999

In [7]:
Session = sessionmaker(bind=m.engine)
ses = Session()

In [8]:
output = widgets.Output()

names = [result.common_name for result in ses.query(m.NCCA.common_name).distinct()]

dropdown_names = widgets.Dropdown(options = names, description='Name')


def dropdown_names_eventhandler(change):
    output.clear_output()

    query = ses.query(m.NCCA).filter(m.NCCA.common_name == dropdown_names.value).statement
    df_out = pd.read_sql(query, m.engine)
    df_out = df_out.sort_values(by='station')
    
    with output:
        display(df_out)
    
dropdown_names.observe(dropdown_names_eventhandler, names='value')

acenthe_min = ses.query(func.min(m.NCCA.ACENTHE)).scalar()
acenthe_max = ses.query(func.max(m.NCCA.ACENTHE)).scalar()

print(acenthe_min)
print(acenthe_max)
acenthe_range = widgets.FloatRangeSlider(
    value=[0, 100.0],
    min=acenthe_min,
    max=acenthe_max,
    step=0.1,
    description='Acenthe range [ng/wet g]',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

query = ses.query(m.NCCA).statement
df = pd.read_sql(query, m.engine)

def acenthe_range_eventhandler(change):
    output.clear_output()
    
    df_out = df[(df['common_name']==dropdown_names.value) & (df['ACENTHE']>=acenthe_range.value[0]) & (df['ACENTHE']<=acenthe_range.value[1])]
    df_out = df_out.sort_values(by='station')
    
    with output:
        display(df_out)

acenthe_range.observe(acenthe_range_eventhandler, names='value')
        
display(dropdown_names)
display(acenthe_range)
display(output)

0.03400
100.00000


Dropdown(description='Name', options=('BROWN BULLHEAD', 'WINTER FLOUNDER', 'ATLANTIC TOMCOD', 'WHITE CATFISH',…

FloatRangeSlider(value=(0.034, 100.0), description='Acenthe range [ng/wet g]', min=0.034, readout_format='.1f'…

Output()